eksik sonuc verdi.

yapılan işlemler onucu skor yükselmedi. Lokasyon, ve keyword gibi değişkenler concatlanıp bakılacak.
Skoru yükselten asıl kaynağın model kurulması ve çalıştırılması olduğu anlaşıldı....

In [172]:
import numpy as np
import pandas as pd
import operator
from wordcloud import STOPWORDS
import string

pd.set_option('display.max_rows', 5000000)
pd.set_option('display.max_columns', 5000000)
pd.set_option('display.width', 10000000)
pd.set_option('display.max_colwidth', 4000)

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv') 

In [3]:
import re
def clean(tweet): 
    
    # Special characters
   
   
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
        
    # ... and ..
    tweet = tweet.replace('...', ' ... ')
    if '...' not in tweet:
        tweet = tweet.replace('..', ' ... ')      
        
    # Acronyms
    tweet = re.sub(r"MH370", "Malaysia Airlines Flight 370", tweet)
    tweet = re.sub(r"mÌ¼sica", "music", tweet)
    tweet = re.sub(r"okwx", "Oklahoma City Weather", tweet)
    tweet = re.sub(r"arwx", "Arkansas Weather", tweet)    
    tweet = re.sub(r"gawx", "Georgia Weather", tweet)  
    tweet = re.sub(r"scwx", "South Carolina Weather", tweet)  
    tweet = re.sub(r"cawx", "California Weather", tweet)
    tweet = re.sub(r"tnwx", "Tennessee Weather", tweet)
    tweet = re.sub(r"azwx", "Arizona Weather", tweet)  
    tweet = re.sub(r"alwx", "Alabama Weather", tweet)
    tweet = re.sub(r"wordpressdotcom", "wordpress", tweet)    
    tweet = re.sub(r"usNWSgov", "United States National Weather Service", tweet)
    tweet = re.sub(r"Suruc", "Sanliurfa", tweet)   
    
    # Grouping same words without embeddings
    tweet = re.sub(r"Bestnaijamade", "bestnaijamade", tweet)
    tweet = re.sub(r"SOUDELOR", "Soudelor", tweet)
    
    return tweet


df_train['text_cleaned'] = df_train['text'].apply(lambda s : clean(s))
df_test['text_cleaned'] = df_test['text'].apply(lambda s : clean(s))





In [4]:
df_mislabeled = df_train.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
df_mislabeled.index.tolist()

['like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit',
 'Hellfire! We don\x89Ûªt even want to think about it or mention it so let\x89Ûªs not do anything that leads to it #islam!',
 "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'",
 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!',
 'To fight bioterrorism sir.',
 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE',
 '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption',
 '#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect',
 'He came to a land which was engulfed in tribal war and turned it into a land 

In [5]:
df_train['target_relabeled'] = df_train['target'].copy() 

df_train.loc[df_train['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'To fight bioterrorism sir.', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "Caution: breathing may be hazardous to your health.", 'target_relabeled'] = 1
df_train.loc[df_train['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target_relabeled'] = 0

In [6]:
# word_count
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

# url_count
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# punctuation_count
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split( df_train[["text",'text_cleaned', "word_count", "unique_word_count","stop_word_count","mean_word_length","char_count","punctuation_count"]]
                                                     , df_train["target_relabeled"], test_size=0.2,
                                                       random_state = 1)

In [78]:
train_x1 = train_x["text_cleaned"]
train_x2 = train_x["word_count"]
train_x3 = train_x["unique_word_count"]
train_x4 = train_x["stop_word_count"]
train_x5 = train_x["mean_word_length"]
train_x6 = train_x["char_count"]
train_x7 = train_x["punctuation_count"]

test_x1 = test_x["text_cleaned"]
test_x2 = test_x["word_count"]
test_x3 = test_x["unique_word_count"]
test_x4 = test_x["stop_word_count"]
test_x5 = test_x["mean_word_length"]
test_x6 = test_x["char_count"]
test_x7 = test_x["punctuation_count"]

In [79]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit_transform(train_x1)


<6090x14841 sparse matrix of type '<class 'numpy.float64'>'
	with 79291 stored elements in Compressed Sparse Row format>

In [80]:
x_train_tf_idf_word1 = tf_idf_word_vectorizer.transform(train_x1)

In [81]:
from scipy.sparse import coo_matrix, hstack

In [82]:
x_train_tf_idf_word2 = coo_matrix(train_x2)
x_train_tf_idf_word2 = x_train_tf_idf_word2.reshape(6090,1)

In [83]:
x_train_tf_idf_word3 = coo_matrix(train_x3)
x_train_tf_idf_word3 = x_train_tf_idf_word3.reshape(6090,1)

In [84]:
x_train_tf_idf_word4 = coo_matrix(train_x4)
x_train_tf_idf_word4 = x_train_tf_idf_word4.reshape(6090,1)

In [85]:
x_train_tf_idf_word5 = coo_matrix(train_x5)
x_train_tf_idf_word5 = x_train_tf_idf_word5.reshape(6090,1)

In [86]:
x_train_tf_idf_word6 = coo_matrix(train_x6)
x_train_tf_idf_word6 = x_train_tf_idf_word6.reshape(6090,1)

In [87]:
x_train_tf_idf_word7 = coo_matrix(train_x7)
x_train_tf_idf_word7 = x_train_tf_idf_word7.reshape(6090,1)

In [88]:
x_train_tf_idf_word = hstack([x_train_tf_idf_word1,x_train_tf_idf_word2,x_train_tf_idf_word3, x_train_tf_idf_word4, x_train_tf_idf_word5, x_train_tf_idf_word6, x_train_tf_idf_word7])

In [89]:
x_train_tf_idf_word = x_train_tf_idf_word.toarray()

In [90]:
type(x_train_tf_idf_word)

numpy.ndarray

In [91]:
x_test_tf_idf_word1 = tf_idf_word_vectorizer.transform(test_x1)

In [92]:
x_test_tf_idf_word2 = coo_matrix(test_x2)
x_test_tf_idf_word2 = x_test_tf_idf_word2.reshape(1523,1)

In [93]:
x_test_tf_idf_word3 = coo_matrix(test_x3)
x_test_tf_idf_word3 = x_test_tf_idf_word3.reshape(1523,1)

In [94]:
x_test_tf_idf_word4 = coo_matrix(test_x4)
x_test_tf_idf_word4 = x_test_tf_idf_word4.reshape(1523,1)

In [95]:
x_test_tf_idf_word5 = coo_matrix(test_x5)
x_test_tf_idf_word5 = x_test_tf_idf_word5.reshape(1523,1)

In [96]:
x_test_tf_idf_word6 = coo_matrix(test_x6)
x_test_tf_idf_word6 = x_test_tf_idf_word6.reshape(1523,1)

In [97]:
x_test_tf_idf_word7 = coo_matrix(test_x7)
x_test_tf_idf_word7 = x_test_tf_idf_word7.reshape(1523,1)

In [98]:
x_test_tf_idf_word = hstack([x_test_tf_idf_word1, x_test_tf_idf_word2,x_test_tf_idf_word3,x_test_tf_idf_word4,x_test_tf_idf_word5,x_test_tf_idf_word6,x_test_tf_idf_word7])

In [99]:
x_test_tf_idf_word = x_test_tf_idf_word.toarray()

In [100]:
type(x_test_tf_idf_word)

numpy.ndarray

In [101]:
test1 = df_test["text_cleaned"]
test1 = tf_idf_word_vectorizer.transform(test1)

In [102]:
np.shape(test1)

(3263, 14841)

In [103]:
test2 = df_test["word_count"]
test2 = coo_matrix(test2)
test2 = test2.reshape(3263,1)

In [104]:
test3 = df_test["unique_word_count"]
test3 = coo_matrix(test3)
test3 = test3.reshape(3263,1)

In [105]:
test4 = df_test["stop_word_count"]
test4 = coo_matrix(test4)
test4 = test4.reshape(3263,1)

In [106]:
test5 = df_test["mean_word_length"]
test5 = coo_matrix(test5)
test5 = test5.reshape(3263,1)

In [107]:
test6 = df_test["char_count"]
test6 = coo_matrix(test6)
test6 = test6.reshape(3263,1)

In [108]:
test7 = df_test["punctuation_count"]
test7 = coo_matrix(test7)
test7 = test7.reshape(3263,1)

In [109]:
test = hstack([test1, test2,test3,test4,test5,test6,test7])


In [110]:
test = test.toarray()

In [111]:
type(test)

numpy.ndarray

In [120]:
from keras.utils import to_categorical
from keras import models
from keras import layers
import numpy as np


from keras.models import Sequential

# Input - Layer
model = Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(14847, )))
# Hidden - Layers
model.add(layers.Dense(25, activation = "relu"))
model.add(layers.Dense(5, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 50)                742400    
_________________________________________________________________
dense_17 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_18 (Dense)             (None, 5)                 130       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 6         
Total params: 743,811
Trainable params: 743,811
Non-trainable params: 0
_________________________________________________________________


In [121]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [123]:
results = model.fit(
 x_train_tf_idf_word, train_y,
 epochs= 11,
 batch_size = 150,
 validation_data = (x_test_tf_idf_word, test_y)
)

Train on 6090 samples, validate on 1523 samples
Epoch 1/11
6090/6090 [==============================] - 2s 323us/step - loss: 0.6716 - accuracy: 0.6061 - val_loss: 0.6466 - val_accuracy: 0.5975
Epoch 2/11
6090/6090 [==============================] - 2s 251us/step - loss: 0.6242 - accuracy: 0.6772 - val_loss: 0.6035 - val_accuracy: 0.6737
Epoch 3/11
6090/6090 [==============================] - 2s 250us/step - loss: 0.5612 - accuracy: 0.7475 - val_loss: 0.5550 - val_accuracy: 0.7367
Epoch 4/11
6090/6090 [==============================] - 2s 250us/step - loss: 0.4845 - accuracy: 0.8084 - val_loss: 0.4955 - val_accuracy: 0.7853
Epoch 5/11
6090/6090 [==============================] - 2s 259us/step - loss: 0.4037 - accuracy: 0.8499 - val_loss: 0.4563 - val_accuracy: 0.7965
Epoch 6/11
6090/6090 [==============================] - 2s 249us/step - loss: 0.3246 - accuracy: 0.8893 - val_loss: 0.4407 - val_accuracy: 0.8011
Epoch 7/11
6090/6090 [==============================] - 2s 250us/step - loss

In [137]:
y_pred_test = model.predict(x_test_tf_idf_word)

In [160]:
np.core.arrayprint._line_width = 1800000000

In [147]:
np.set_printoptions(threshold=np.inf)

In [178]:
y_pred_test = model.predict(x_test_tf_idf_word)

In [180]:
y_pred_test = model.predict(x_test_tf_idf_word)
y_pred_test = (y_pred_test > 0.5)
y_pred_test = pd.DataFrame(data = y_pred_test, index = range(1523), columns=['target'])
y_pred_test["target"] = y_pred_test["target"].astype(int)

In [181]:
print(y_pred_test)

      target
0          1
1          1
2          0
3          0
4          0
5          0
6          0
7          0
8          1
9          0
10         0
11         0
12         0
13         0
14         1
15         0
16         0
17         1
18         1
19         0
20         0
21         0
22         1
23         0
24         1
25         1
26         1
27         0
28         0
29         0
30         0
31         1
32         0
33         1
34         1
35         0
36         1
37         0
38         0
39         0
40         1
41         1
42         1
43         0
44         1
45         1
46         0
47         0
48         0
49         1
50         0
51         0
52         1
53         0
54         1
55         0
56         1
57         0
58         1
59         0
60         0
61         0
62         1
63         1
64         1
65         0
66         0
67         0
68         0
69         0
70         1
71         1
72         0
73         1
74         0
75         0

In [ ]:
print(x_test_tf_idf_word)

In [176]:
test_x1 = test_x[["text"]]

In [177]:
print(test_x1)

                                                                                                                                                       text
3228          Goulburn man Henry Van Bilsen missing: Emergency services are searching for a Goulburn man who disappeared from hisÛ_ http://t.co/z99pKJzTRp
3706             The things we fear most in organizations--fluctuations disturbances imbalances--are the primary sources of creativity. - Margaret Wheatley
6957                                                                                                                                @tsunami_esh ?? hey Esh
2887           @POTUS you until you drown by water entering the lungs. You being alive has caused this great country to fall to shit because you're a pussy
7464                                                                                                    Crawling in my skin\nThese wounds they will not hea
2539                                                            

In [118]:
y_pred = model.predict(test)
y_pred = (y_pred > 0.5)
y_pred1 = pd.DataFrame(data = y_pred, index = range(3263), columns=['target'])
y_pred1["target"] = y_pred1["target"].astype(int)

In [136]:
from sklearn.metrics import classification_report,confusion_matrix
cm = confusion_matrix(test_y, y_pred_test)
print(cm)

[[683 203]
 [141 496]]


In [119]:
submission = pd.read_csv('sample_submission.csv') 
submission= submission[["id"]]
final_submission=pd.concat([submission,y_pred1],axis=1)
final_submission.to_csv('12062020_15.csv',index=False)


In [156]:
final_submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [51]:
np.shape(final_submission)

(3263, 2)

In [270]:
np.shape(x_train_tf_idf_word)

(5709, 7123)